In [53]:
import os
import fitz
from tqdm import tqdm

In [54]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    elif rate_1 in possible_rate:
        return rate_1
    elif rate_2 in possible_rate:
        return rate_2
    else :
        return 'NULL'
            

In [55]:
zip_path = r'..\file\NMR'
count_true = 0
possible_rate = ['Buy', 'Overweight', 'Neutral', 'Underweight', 'Sell']
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    rate_1 = None
    rate_2 = None
    rate = None
    try:
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)
            rect = page.rect
            page_check_source = doc.load_page(-1)
            text_check_source = page_check_source.get_text()
            if 'Nomura Internationa' in text_check_source :
                # 提取評價的第一種方法
                try:
                    clip_old_report_1 = fitz.Rect(410, 150, rect.width, 500)
                    text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
                    text_old_report_1 = text_old_report_1.split('Rating')[1].strip()
                    rate_1 = text_old_report_1.split('\n')[1].strip()
                except:
                    rate_1 = None
                    # 提取評價的第二種方法
                try:
                    clip_old_report_2 = fitz.Rect(470, 170, rect.width, 200)
                    rate_2 = page.get_text(clip=clip_old_report_2).strip()
                except:
                    rate_2 = None
                # if rate_1 == rate_2 and rate_1 not in possible_rate:
                #     possible_rate.append(rate_1)
                rate = check_rate_function(rate_1, rate_2, possible_rate)
            else:
                # False
                print(f'{directory_path} is not posted by hsbc')
    except:
        print(directory_path, 'fail')
    if rate == 'NULL' :
        print('case1', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
    elif rate == 'wrong':
        count_true += 1
        # print(directory_path, rate,
        #       f'text_check_new_report={text_check_new_report}, text_check_old_fast_report={text_check_old_fast_report}')
    elif rate != None :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    else :
        print('case2', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

100%|██████████| 17/17 [00:00<00:00, 40.14it/s]

共17件, 成功17件, 失敗0件, 成功率:100.0%


In [56]:
import fitz
doc = fitz.open(r'..\file\NMR\1590_亞德克_NMR_NEUTRAL.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [57]:
# 檢查是否由第一金投顧出版
page_check_source = doc.load_page(-1)
text_check_source = page_check_source.get_text()
if 'Nomura Internationa' in text_check_source :
    print('True')
text_check_source

True


'Recommending Trades in Securities to Customers and/or other applicable laws or regulations in Taiwan, you are prohibited to provide the reports to \nothers (including but not limited to related parties, affiliated companies and any other third parties) or engage in any activities in connection with the \nreports which may involve conflicts of interests. INFORMATION ON SECURITIES / INSTRUMENTS NOT EXECUTABLE BY NOMURA \nINTERNATIONAL (HONG KONG) LTD., TAIPEI BRANCH IS FOR INFORMATIONAL PURPOSES ONLY AND IS NOT BE CONSTRUED AS A \nRECOMMENDATION OR A SOLICITATION TO TRADE IN SUCH SECURITIES / INSTRUMENTS.\nThis material may not be distributed in Indonesia or passed on within the territory of the Republic of Indonesia or to persons who are Indonesian \ncitizens (wherever they are domiciled or located) or entities of or residents in Indonesia in a manner which constitutes a public offering under the \nlaws of the Republic of Indonesia. The securities mentioned in this document may not be 

In [58]:
# 檢查版本
clip_check_report = fitz.Rect(350, 0, rect.width, 250)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_report)
pix.save("page-clip_check_report.png")
text_check_report = page.get_text(clip=clip_check_report)
if 'Equities' in text_check_report :
    print('Equities')
else :
    print('NULL')
text_check_report

NULL


'Global Markets Research\n5 February 2023\nroved\nent in daily \nRating\nRemains\nNeutral\nTarget price\nRemains\nTWD 747.00\nClosing price\n03 February 2023\nTWD \n'

In [59]:
# 報告為新版個股報告 提取評價的第一種方法
try:
    clip_old_report_1 = fitz.Rect(410, 150, rect.width, 500)
    text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
    pix.save("page-clip_new_report_1.png")
    text_old_report_1 = text_old_report_1.split('Rating')[1].strip()
    rate_1 = text_old_report_1.split('\n')[1].strip()
except:
    rate_1 = None
rate_1

'Neutral'

In [60]:
# 報告為新版個股報告 提取評價的第二種方法
try:
    clip_old_report_2 = fitz.Rect(470, 170, rect.width, 200)
    rate_2 = page.get_text(clip=clip_old_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_2)
    pix.save("page-clip_new_report_2.png")
except:
    rate_2 = None
rate_2

'Neutral'